In [1]:
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import albumentations
import albumentations.pytorch 
import cv2
import torch.nn as nn
import copy
from barbar import Bar

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark=True
    torch.backends.cudnn.deterministic=False  

In [4]:
#train_pure = '/kaggle/input/breast-256/gcn_256/Train_gcn/*'
val_pure =  'gcn_256/Val_gcn/*'

In [5]:
import glob
#train_files = glob.glob(train_pure)
val_files = glob.glob(val_pure)
len(val_files)#6229

1557

In [6]:
val_files[0]

'gcn_256/Val_gcn/SOB_M_PC-15-190EF-200-001.png'

In [7]:
x=val_files[0]
x.split('-')[-2]

'200'

In [8]:
val_40=[]
val_100=[]
val_200=[]
val_400=[]
for i in val_files:
    parts=i.split('-')
    if (parts[-2])== "40":
        val_40.append(i)
    elif (parts[-2])== "100":
        val_100.append(i)
    elif (parts[-2])== "200":
        val_200.append(i)
    elif (parts[-2])== "400":
        val_400.append(i)
print(len(val_40),len(val_100),len(val_200),len(val_400))

373 440 386 358


In [9]:
import albumentations
import albumentations.pytorch 
class My_data(Dataset):
    def __init__(self, data, transforms=None):
        self.image_list = data
        self.data_len = len(self.image_list)
        self.transforms = transforms
        self.eicls = ["A", "F", "TA", "PT", "DC", "LC", "MC", "PC"]
        
    def __getitem__(self, index):
        current_image_path = self.image_list[index]
        im_as_im = cv2.imread(current_image_path)
        im_as_im = cv2.cvtColor(im_as_im, cv2.COLOR_BGR2RGB)

        # Perform label encoding for multi-label classification
        parts = current_image_path.split('_')[-1].split('-')
        if parts[2]=="13412":
            labels =[0,0,0,0,1,1,0,0]
        else:
            labels = [int(label == parts[0]) for label in self.eicls]       
        labels = torch.tensor(labels)

        if self.transforms is not None:
            augmented = self.transforms(image=im_as_im)
            im_as_im = augmented['image']

        return (im_as_im, labels)

    def __len__(self):
        return self.data_len             
 
   

In [10]:
transform = {
    'train': albumentations.Compose([
    albumentations.Resize(256, 256),     
    albumentations.OneOf([
                          albumentations.HorizontalFlip(),
                          albumentations.RandomRotate90(),
                          albumentations.VerticalFlip(),
                          albumentations.GaussianBlur(),
                          albumentations.NoOp()
    ], p=1),
    albumentations.Normalize(mean=(0.787, 0.625, 0.765),
                       std=(0.105, 0.138, 0.089), p=1),
    albumentations.pytorch.transforms.ToTensorV2()]),
    
    'valid': albumentations.Compose([
    albumentations.Resize(256, 256),     
    albumentations.Normalize(mean=(0.786, 0.623, 0.766),
                       std=(0.105, 0.138, 0.089), p=1),
    albumentations.pytorch.transforms.ToTensorV2()]),}
    

valid=My_data(val,transforms=transform['valid'])
valid_dataloader=  torch.utils.data.DataLoader(dataset=valid,batch_size=1,shuffle=False,num_workers=4,
                                               pin_memory=True )  

In [11]:
valid_40=My_data(val_40,transforms=transform['valid'])
valid_dataloader1=  torch.utils.data.DataLoader(dataset=valid_40,batch_size=1,shuffle=False,num_workers=4,
                                               pin_memory=True )  

In [12]:
valid_100=My_data(val_100,transforms=transform['valid'])
valid_dataloader2=  torch.utils.data.DataLoader(dataset=valid_100,batch_size=1,shuffle=False,num_workers=4,
                                               pin_memory=True )  

In [13]:
valid_200=My_data(val_200,transforms=transform['valid'])
valid_dataloader3=  torch.utils.data.DataLoader(dataset=valid_200,batch_size=1,shuffle=False,num_workers=4,
                                               pin_memory=True )  

In [14]:
valid_400=My_data(val_400,transforms=transform['valid'])
valid_dataloader4=  torch.utils.data.DataLoader(dataset=valid_400,batch_size=1,shuffle=False,num_workers=4,
                                               pin_memory=True )  

In [15]:
import torch
class_samples = [367, 803, 456, 370, 2763, 492, 629, 449]  # Number of samples in each class
total_samples = sum(class_samples)
samples=total_samples/len(class_samples)
class_weights = [samples / (s + 1e-8) for s in class_samples]
class_weights = torch.tensor(class_weights)
print(class_weights)

tensor([2.1557, 0.9852, 1.7349, 2.1382, 0.2863, 1.6080, 1.2578, 1.7620])


In [16]:
import torch
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, class_weights=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.class_weights = class_weights

    def forward(self, logits, labels):
        probs = torch.sigmoid(logits)
        ce_loss = nn.BCELoss()(probs, labels)
        weight = (1 - probs).pow(self.gamma)
        loss = ce_loss  # Initialize loss with cross-entropy loss
        if self.class_weights is not None:
            weight = weight * self.class_weights
            loss = loss * weight
        return loss

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import timm
from timm.models import create_model
from timm.data import create_transform
from sklearn.metrics import accuracy_score

# Define device
# Define Swin Transformer v2 model
"""model_name = 'swin_base_patch4_window7_224'
num_classes = 8
model = create_model(
    model_name=model_name,
    pretrained=True,
    num_classes=num_classes,
    drop_rate=0.5,
    drop_path_rate=0.2,
    checkpoint_path=None
)"""
model = timm.create_model(
    'swinv2_tiny_window8_256.ms_in1k',
    pretrained=False,
    features_only=False,
    num_classes = 8,
    drop_path_rate=0.2
)
class_weights=class_weights.to(device)
criterion = FocalLoss(class_weights)
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
checkpoint1 = torch.load('ac98data/eft_tiny.pth')
model.load_state_dict(checkpoint1['model_state_dict'])
model = model.to(device)

/home/kb/anaconda3/envs/pytorch-gpu/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [18]:
#define the function that take model and etc.. and compute val loss and val_accuracy and ypred and y true
def validate(model, dataloader, optimizer, criterion):
    #print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    y_pred = []
    y_true = []
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.float()
            labels = labels.to(device)
            outputs = model(inputs)
            thresholds = [0.5, 0.5, 0.5,0.5,0.5,0.5,0.5,0.5]
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item()*inputs.size(0)
            probs = torch.sigmoid(outputs)
            preds = torch.zeros_like(probs)
           
            y_pred.extend(preds)
            y_true.extend(labels)
            
            #print(preds,labels)
            for i, threshold in enumerate(thresholds):
                preds[:, i] = (probs[:, i] >= threshold).float()
            val_running_correct += (preds == labels).all(dim=1).float().sum()
            
        
    val_loss = val_running_loss/len(dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(dataloader.dataset)        
    return val_loss, val_accuracy,y_pred,y_true

In [22]:
from sklearn.metrics import confusion_matrix,recall_score,f1_score,precision_score,matthews_corrcoef,accuracy_score
import seaborn as sn
import pandas as pd
val_loss1, val_accuracy1,y_pred1,y_true1 = validate(model,valid_dataloader1, optimizer, criterion)
#converting cuda list to cpu list and subsequently converting numpy
#to work with sklearn.matrics
y_pred1 = [tensor.cpu().numpy() for tensor in y_pred1]
y_true1=  [tensor.cpu().numpy() for tensor in y_true1]
#y_pred= torch.tensor(y_pred, device = 'cpu').detach().numpy() 
#y_true= torch.tensor(y_true, device = 'cpu').detach().numpy() 
accuracy =accuracy_score (y_true1, y_pred1)       
recall=recall_score(y_true1, y_pred1, average='weighted')
f1=f1_score(y_true1, y_pred1, average='weighted')
precision=precision_score(y_true1, y_pred1, labels=None, pos_label=1, average='weighted')
#mcc=matthews_corrcoef(y_true1, y_pred1, sample_weight=None)  
print("accuracy_40=",accuracy ,"recall_40=",recall,"F1-measure_40=",f1,"precision_40=" ,precision)
print(accuracy , "&" ,recall,  "&" ,f1  ,"&" ,precision)
#print("accuracy=",accuracy ,"recall=",recall,"F1-measure=",f1,"precision=" ,precision,"MCC=",mcc)

accuracy_40= 0.9946380697050938 recall_40= 0.994750656167979 F1-measure_40= 0.996022681151058 precision_40= 0.9973930623536923
0.9946380697050938 & 0.994750656167979 & 0.996022681151058 & 0.9973930623536923


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_true1, y_pred1, digits=3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        20
           1      1.000     1.000     1.000        59
           2      1.000     1.000     1.000        29
           3      1.000     1.000     1.000        17
           4      0.993     1.000     0.997       147
           5      1.000     0.973     0.986        37
           6      1.000     1.000     1.000        47
           7      1.000     0.960     0.980        25

   micro avg      0.997     0.995     0.996       381
   macro avg      0.999     0.992     0.995       381
weighted avg      0.997     0.995     0.996       381
 samples avg      0.997     0.996     0.996       381



In [24]:
from sklearn.metrics import confusion_matrix,recall_score,f1_score,precision_score,matthews_corrcoef,accuracy_score
import seaborn as sn
import pandas as pd
val_loss2, val_accuracy2,y_pred2,y_true2 = validate(model,valid_dataloader2, optimizer, criterion)
#converting cuda list to cpu list and subsequently converting numpy
#to work with sklearn.matrics
y_pred2 = [tensor.cpu().numpy() for tensor in y_pred2]
y_true2=  [tensor.cpu().numpy() for tensor in y_true2]
#y_pred= torch.tensor(y_pred, device = 'cpu').detach().numpy() 
#y_true= torch.tensor(y_true, device = 'cpu').detach().numpy() 
accuracy =accuracy_score (y_true2, y_pred2)       
recall=recall_score(y_true2, y_pred2, average='weighted')
f1=f1_score(y_true2, y_pred2, average='weighted')
precision=precision_score(y_true2, y_pred2, labels=None, pos_label=1, average='weighted')
#mcc=matthews_corrcoef(y_true1, y_pred1, sample_weight=None)  
print("accuracy_100=",accuracy ,"recall_100=",recall,"F1-measure_100=",f1,"precision_100=" ,precision)
#print("accuracy=",accuracy ,"recall=",recall,"F1-measure=",f1,"precision=" ,precision,"MCC=",mcc)
print(accuracy , "&" ,recall,  "&" ,f1  ,"&" ,precision)

accuracy_100= 0.9863636363636363 recall_100= 0.9865168539325843 F1-measure_100= 0.985565744665851 precision_100= 0.984940923480868
0.9863636363636363 & 0.9865168539325843 & 0.985565744665851 & 0.984940923480868


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_true2, y_pred2, digits=3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        22
           1      0.984     1.000     0.992        61
           2      1.000     1.000     1.000        27
           3      1.000     0.952     0.976        21
           4      0.995     0.985     0.990       200
           5      0.902     0.974     0.937        38
           6      0.979     0.979     0.979        47
           7      1.000     1.000     1.000        29

   micro avg      0.984     0.987     0.985       445
   macro avg      0.982     0.986     0.984       445
weighted avg      0.985     0.987     0.986       445
 samples avg      0.986     0.986     0.986       445



In [26]:
from sklearn.metrics import confusion_matrix,recall_score,f1_score,precision_score,matthews_corrcoef,accuracy_score
import seaborn as sn
import pandas as pd
val_loss3, val_accuracy3,y_pred3,y_true3 = validate(model,valid_dataloader3, optimizer, criterion)
#converting cuda list to cpu list and subsequently converting numpy
#to work with sklearn.matrics
y_pred3 = [tensor.cpu().numpy() for tensor in y_pred3]
y_true3=  [tensor.cpu().numpy() for tensor in y_true3]
#y_pred= torch.tensor(y_pred, device = 'cpu').detach().numpy() 
#y_true= torch.tensor(y_true, device = 'cpu').detach().numpy() 
accuracy =accuracy_score (y_true3, y_pred3)       
recall=recall_score(y_true3, y_pred3, average='weighted')
f1=f1_score(y_true3, y_pred3, average='weighted')
precision=precision_score(y_true3, y_pred3, labels=None, pos_label=1, average='weighted')
#mcc=matthews_corrcoef(y_true1, y_pred1, sample_weight=None)  
print("accuracy_200=",accuracy ,"recall_200=",recall,"F1-measure_200=",f1,"precision_200=" ,precision)
#print("accuracy=",accuracy ,"recall=",recall,"F1-measure=",f1,"precision=" ,precision,"MCC=",mcc)
print(accuracy , "&" ,recall,  "&" ,f1  ,"&" ,precision)

accuracy_200= 0.9766839378238342 recall_200= 0.9770992366412213 F1-measure_200= 0.9771231241250357 precision_200= 0.9778944211524738
0.9766839378238342 & 0.9770992366412213 & 0.9771231241250357 & 0.9778944211524738


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_true3, y_pred3, digits=3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        16
           1      0.957     1.000     0.978        45
           2      1.000     1.000     1.000        27
           3      1.000     0.917     0.957        24
           4      0.994     0.972     0.983       181
           5      0.941     0.941     0.941        34
           6      0.929     1.000     0.963        39
           7      0.964     1.000     0.982        27

   micro avg      0.977     0.977     0.977       393
   macro avg      0.973     0.979     0.975       393
weighted avg      0.978     0.977     0.977       393
 samples avg      0.977     0.977     0.977       393



In [28]:
from sklearn.metrics import confusion_matrix,recall_score,f1_score,precision_score,matthews_corrcoef,accuracy_score
import seaborn as sn
import pandas as pd
val_loss4, val_accuracy4,y_pred4,y_true4 = validate(model,valid_dataloader4, optimizer, criterion)
#converting cuda list to cpu list and subsequently converting numpy
#to work with sklearn.matrics
y_pred4 = [tensor.cpu().numpy() for tensor in y_pred4]
y_true4=  [tensor.cpu().numpy() for tensor in y_true4]
#y_pred= torch.tensor(y_pred, device = 'cpu').detach().numpy() 
#y_true= torch.tensor(y_true, device = 'cpu').detach().numpy() 
accuracy =accuracy_score (y_true4, y_pred4)       
recall=recall_score(y_true4, y_pred4, average='weighted')
f1=f1_score(y_true4, y_pred4, average='weighted')
precision=precision_score(y_true4, y_pred4, labels=None, pos_label=1, average='weighted')
#mcc=matthews_corrcoef(y_true1, y_pred1, sample_weight=None)  
print("accuracy_400=",accuracy ,"recall_400=",recall,"F1-measure_400=",f1,"precision_400=" ,precision)
#print("accuracy=",accuracy ,"recall=",recall,"F1-measure=",f1,"precision=" ,precision,"MCC=",mcc)
print(accuracy , "&" ,recall,  "&" ,f1  ,"&" ,precision)

accuracy_400= 0.9720670391061452 recall_400= 0.9778393351800554 F1-measure_400= 0.9749580835916278 precision_400= 0.9727688890477291
0.9720670391061452 & 0.9778393351800554 & 0.9749580835916278 & 0.9727688890477291


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true4, y_pred4, digits=3))

In [ ]:
98.64 & 98.65 & 98.56 & 98.49
97.67 & 97.71 & 97.71 & 97.79
97.21 & 97.78 & 97.50 & 97.28